TITANIC DATASET - ML - RANDOM FOREST
====================================

In [2]:
%load_ext autoreload
%autoreload 2

import mlflow
import pickle
import pandas as pd 
import numpy as np

from datetime import datetime
from functions import get_metrics

from matplotlib import pyplot as plt 
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Configs

In [3]:
EXPERIMENT_TITLE = "Titanic Dataset Analyzes"

run_description = """
### Descrição

Implementação usando [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
"""

tags = {
    "date": datetime.now(),
    "author": "Pablo Veinberg",
    "version": 1.0,
    "envoronment": "local",
    "mlflow.note.content": run_description,
    "mlflow.runName": "Random Forest",
    "data_source": "./../datasets/silver/train-encoded-not-normalize.parquet",
    "train_test_dataset": "./../datasets/silver/titanic-train-test-data.pkl"
}

params = {
    "token": np.random.randint(10_000,high=99_000),
    "model_params": {
        "criterion": ["gini", "entropy", "log_loss"],
        "max_features": ["sqrt", "log2", None]
    }
}


In [4]:
dataset = pd.read_parquet(tags['data_source'])
mlflow_dataset = mlflow.data.from_pandas(dataset, \
                                            source=tags['data_source'], \
                                            name="Titanic Dataset")

/home/pablo_veinberg/.local/lib/python3.10/site-packages/mlflow/data/dataset_source_registry.py:143: UserWarning: The specified dataset source can be interpreted in multiple ways: LocalArtifactDatasetSource, LocalArtifactDatasetSource. MLflow will assume that this is a LocalArtifactDatasetSource source.
  return _dataset_source_registry.resolve(


In [5]:
with open('./../datasets/silver/titanic-train-test-data.pkl', 'rb') as file:
    X_train, X_test, y_train, y_test = pickle.load(file)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 8), (179, 8), (712,), (179,))

## Process

In [6]:
mlflow.set_experiment(EXPERIMENT_TITLE)

with mlflow.start_run():

    # Settings
    mlflow.log_input(mlflow_dataset)
    mlflow.set_tags(tags)
    mlflow.log_params(params)

    grid = GridSearchCV(estimator=RandomForestClassifier(), \
                        param_grid=params["model_params"])
    grid.fit(X_train, y_train)

    model = RandomForestClassifier(**grid.best_params_)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    plot_confusion_matrix_path = f"./../results/random_forest_{params['token']}.png"
    metrics = get_metrics(y_test, y_pred, plot_confusion_matrix_path, "Matrix Confusion - Titanic Dataset")
    
    mlflow.log_param("best_params", grid.best_params_)
    mlflow.log_metrics(metrics)
    mlflow.log_artifact(plot_confusion_matrix_path)
    
mlflow.end_run()

Traceback (most recent call last):
  File "/home/pablo_veinberg/.local/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 302, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/home/pablo_veinberg/.local/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 395, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/home/pablo_veinberg/.local/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 1303, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/home/pablo_veinberg/.local/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 1296, in _read_helper
    result = read_yaml(root, file_name)
  File "/home/pablo_veinberg/.local/lib/python3.10/site-packages/mlflow/utils/file_utils.py", line 303, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' does not exist.")
mlflow.e

/home/pablo_veinberg/.local/lib/python3.10/site-packages/mlflow/data/pandas_dataset.py:134: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  return _infer_schema(self._df)


In [7]:
# Survival|0 = No, 1 = Yes
cm = confusion_matrix(y_test, y_pred)
cm

array([[98,  7],
       [ 7, 67]])

In [8]:
class_report = classification_report(y_test, y_pred, 
                           output_dict=True)

results = class_report['macro avg']


# precisão indica quão bom o modelo é para detectar positivos

labels = ['não sobreviventes', 'sobreviventes']

for key, value in enumerate(labels):
    print(f"O modelo é bom em {round(class_report[str(key)]['precision'] * 100 ,2)}% para acertar {value}")
    print(f"Quando acerta, está correto num {round(class_report[str(key)]['recall'] * 100 ,2)}%")
    print(f"Esta informação foi baseada em {round(class_report[str(key)]['support'],0)} registros")
    print("="*50)

# print(f"O modelo é bom em {round(class_report['0']['precision'] * 100 ,2)}% para acertar não sobreviventes")
# print(f"Quando acerta, está correto num {round(class_report['0']['recall'] * 100 ,2)}%")


# # recall indica quando a resposta real é sim, quanto o modelo acertou
# print(class_report)

O modelo é bom em 93.33% para acertar não sobreviventes
Quando acerta, está correto num 93.33%
Esta informação foi baseada em 105 registros
O modelo é bom em 90.54% para acertar sobreviventes
Quando acerta, está correto num 90.54%
Esta informação foi baseada em 74 registros


In [9]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93       105
           1       0.91      0.91      0.91        74

    accuracy                           0.92       179
   macro avg       0.92      0.92      0.92       179
weighted avg       0.92      0.92      0.92       179

